In [5]:
from bs4 import BeautifulSoup
import requests
import re

In [6]:
class Emprego:

    def clear(self, txt):
        return str(txt).replace("\n", "").replace("\t", "").replace("\b", "")

    def encode_spaces(self, txt):
        return str(txt).replace(" ","+")

    def fetch_job(self):
        for job_listing in self.job_page.findAll(class_="thumbnail"):
            try:
                self.extract_job_information(job_listing["href"])
            except Exception as erro:
                break

    job_listings = []
    def extract_job_information(self, url):
        link = url
        page = BeautifulSoup(requests.get(url).text, "lxml")
        html = page.find(class_="col-lg-8 conteudo-vaga")

        details = []
        details.append(self.clear(html.h1.span)[17:-16])

        for count, topic in enumerate(html.findAll("p")):
            if(count == 2 or count == 3 or count == 4 or count == 5
               or count == 6):
                    details.append(topic.text)
            elif(count >= 7):
                try:
                    self.check_email_and_phone_validity(details, topic.text)
                except Exception as erro:
                    print("SEM CONTATO", erro)
        jobs = {'title': details[0], 'salary': details[3][8:], 'beneficies': details[4][11:], 'details_job':details[1][19:],
                     'requests': details[2][12:], 'contact': details[6], 'link': link }
        self.job_listings.append(jobs)

    def check_email_and_phone_validity(self, details, txt):
        email_phone_regex_patterns = ["^([1-9]{2}) 9[7-9]{1}[0-9]{3}-[0-9]{4}$",
                  "[a-zA-Z0-9]+[a-zA-Z0-9_.-]+@{1}[a-zA-Z0-9_.-]*\\.+[a-z]{2,4}"]
        for type_index in range(2):
            try:
                details.append(re.findall(email_phone_regex_patterns[type_index], txt)[0])
            except Exception as erro:
                pass

    def navigate_jobs(self, cargo, contador):
        url = "http://empregacampinas.com.br/page/{}/?s={}".format(contador, self.encode_spaces(cargo))
        self.job_page = BeautifulSoup(requests.get(url).text, "lxml")
        self.fetch_job()

    def __str__(self):
        return self.job_page.text


In [12]:
emprego = Emprego()
for page in range(10):
    emprego.navigate_jobs("python", page)
    if(len(emprego.job_listings) <= 0):
        break
    

In [11]:
emprego.job_listings[0]

['ROGRAMADOR DELPHI / NOVA ODESSA – SP / 1 VAGA(S) ',
 ' a combinar',
 ' Convênio Médico, Convênio Odontológico, Cesta Básica, Refeição no local, Vale Transporte.',
 'Desenvolver, testar e manter aplicações utilizando Delphi; Colaborar com a equipe para identificar e solucionar problemas técnicos; Participar do ciclo completo de desenvolvimento de software, desde a concepção até a implementação; Garantir a qualidade e a eficiência do código; Documentar processos e soluções técnicas.',
 'Experiência com desenvolvimento em Delphi; Conhecimento em banco de dados (Postgresql, etc.); Boa capacidade de análise e resolução de problemas; Trabalho em equipe e boa comunicação.',
 'oportunidade.novo.trabalho@gmail.com',
 'https://empregacampinas.com.br/programador-delphi-nova-odessa-sp-1-vagas/']